In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
import matplotlib.pyplot as plt
import contextily as cx
from rasterio.crs import CRS
from pathlib import Path
import pickle
from shapely.geometry import Polygon
from geopandas import gpd
import numpy as np

In [4]:
def calculate_averages(results, df_points):
    
    df_temp = df_points.copy()
    av_cad_hours = []
#     avail_sensors = []
    n_avail_sensors = []
    av_cad_hours += [i.cadence.mean().seconds/3600 if type(i) is not str else np.nan for i in results]
#     avail_sensors += [i.sensor.unique() if type(i) is not str else '' for i in results]
    n_avail_sensors += [len(i.sensor.unique()) if type(i) is not str else np.nan for i in results]
    
            
    df_temp['av_cad_hours'] = results.groupby(['cx','cy']).mean().reset_index(drop=True).cadence.seconds / 3600
#     df_temp['av_cad_hours'] = av_cad_hours
#     df_temp['avail_sensors'] = avail_sensors
    df_temp['n_avail_sensors'] = n_avail_sensors
    
    return df_temp

In [5]:
# Re-grid results to 1x1 degree cells
def regrid(df_temp: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    
    df_temp['lon'] = df_temp.geometry.map(get_lon)
    df_temp['lat'] = df_temp.geometry.map(get_lat)
    
    return df_temp

# Use geodataframe groupby function to aggregate rows to 1 by 1 degree box
def aggregator(df_temp: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    df_ag = df_temp.groupby(['lon', 'lat']).mean().reset_index()
    geom = df_ag.apply(make_box, axis=1)
    df_ag = gpd.GeoDataFrame(df_ag,
                             geometry=geom,
                             crs = CRS.from_epsg(4326))
    return df_ag

In [20]:
output_path = Path('../output_dfs')
dfs = sorted(list(output_path.glob('*.pkl')))
print(dfs)

[PosixPath('../output_dfs/Africa_points_results.pkl'), PosixPath('../output_dfs/Africa_results.pkl'), PosixPath('../output_dfs/Asia_points_results.pkl'), PosixPath('../output_dfs/Asia_results.pkl'), PosixPath('../output_dfs/Europe_points_results.pkl'), PosixPath('../output_dfs/Europe_results.pkl'), PosixPath('../output_dfs/North America_points_results.pkl'), PosixPath('../output_dfs/North_America_results.pkl'), PosixPath('../output_dfs/Oceania_points_results.pkl'), PosixPath('../output_dfs/Oceania_results.pkl'), PosixPath('../output_dfs/South America_points_results.pkl'), PosixPath('../output_dfs/South_America_results.pkl')]


In [52]:
continent_list = ['Africa','Asia','Europe','N_America','Oceania','S_America']
continents = {}
cont_points = {}
for i in range(len(continent_list)):
#     with open(dfs[i * 2], 'rb') as f:
#         cont_points[continent_list[i]] = pickle.load(f)
#     with open(dfs[i * 2 + 1], 'rb') as f:
#         continents[continent_list[i]] = pickle.load(f)
    print(dfs[i * 2],dfs[i * 2 + 1])

../output_dfs/Africa_points_results.pkl ../output_dfs/Africa_results.pkl
../output_dfs/Asia_points_results.pkl ../output_dfs/Asia_results.pkl
../output_dfs/Europe_points_results.pkl ../output_dfs/Europe_results.pkl
../output_dfs/North America_points_results.pkl ../output_dfs/North_America_results.pkl
../output_dfs/Oceania_points_results.pkl ../output_dfs/Oceania_results.pkl
../output_dfs/South America_points_results.pkl ../output_dfs/South_America_results.pkl


In [38]:
df = list(output_path.glob('results.pkl'))

In [29]:
with open(dfs[3], 'rb') as f:
#     cont_points['Asia'] = pickle.load(f)
    continents['Asia'] = pickle.load(f)

In [39]:
with open(df[0],'rb') as f:
    results = pickle.load(f)

In [47]:
results.cx

In [28]:
dfs[3]

PosixPath('../output_dfs/Asia_results.pkl')

In [31]:
df_temp = calculate_averages(continents['Africa'], cont_points['Africa'])

ValueError: Length of values (7) does not match length of index (34878)

In [ ]:
continents['N_America'].groupby(['cx','cy']).mean()

In [32]:
len(continents['Asia'])

40253

In [33]:
len(continents['Africa'])

28448

In [ ]:
testing = continents['Asia'].copy()

In [ ]:
testing.dissolve(by='cx')

In [48]:
continents['Asia']['cy'].unique()

array([38.69894074, 39.71058508, 40.72222942, 41.73387376, 36.67565207,
       37.6872964 , 34.65236339])

In [51]:
continents['Asia'].head(20)

,startTime,geometry,fileID,sensor,cadence,cx,cy
0,2022-01-02 09:09:27.784000+00:00,"POLYGON ((27.11095 37.85939, 25.86283 37.85395...",HLS.S30.T35SMD.2022002T090259.v2.0,sentinel2,NaT,26.543351,38.698941
1,2022-01-03 08:52:13.378000+00:00,"POLYGON ((27.11095 37.85939, 25.86283 37.85395...",HLS.L30.T35SMD.2022003T085213.v2.0,landsat8,0 days 23:42:45.594000,26.543351,39.710585
2,2022-01-04 04:23:20+00:00,"POLYGON ((26.63933 38.44150, 27.04625 40.06287...",S1A_IW_SLC__1SDV_20220104T042320_20220104T0423...,sentinel1,0 days 19:31:06.622000,26.543351,40.722229
3,2022-01-05 16:07:37+00:00,"POLYGON ((25.77838 39.86797, 26.18389 38.24693...",S1A_IW_SLC__1SDV_20220105T160737_20220105T1608...,sentinel1,1 days 11:44:17,26.543351,41.733874
4,2022-01-07 09:09:47.978000+00:00,"POLYGON ((25.86283 37.85395, 27.11095 37.85939...",HLS.S30.T35SMC.2022007T090351.v2.0,sentinel2,1 days 17:02:10.978000,27.547587,36.675652
5,2022-01-10 16:15:46+00:00,"POLYGON ((23.78506 39.58527, 24.18896 37.96430...",S1A_IW_SLC__1SDV_20220110T161546_20220110T1616...,sentinel1,3 days 07:05:58.022000,27.547587,37.687296
6,2022-01-11 04:15:11+00:00,"POLYGON ((28.62529 38.15443, 29.03035 39.77561...",S1A_IW_SLC__1SDV_20220111T041511_20220111T0415...,sentinel1,0 days 11:59:25,27.547587,38.698941
7,2022-01-12 09:09:27.708000+00:00,"POLYGON ((27.11095 37.85939, 25.86283 37.85395...",HLS.S30.T35SMD.2022012T090229.v2.0,sentinel2,1 days 04:54:16.708000,27.547587,39.710585
8,2022-01-16 04:23:19+00:00,"POLYGON ((26.63951 38.44187, 27.04635 40.06288...",S1A_IW_SLC__1SDV_20220116T042319_20220116T0423...,sentinel1,3 days 19:13:51.292000,27.547587,41.733874
9,2022-01-17 09:09:32.294000+00:00,"POLYGON ((27.11095 37.85939, 25.86283 37.85395...",HLS.S30.T35SMD.2022017T090321.v2.0,sentinel2,1 days 04:46:13.294000,28.551822,36.675652


In [18]:
cont_points['Europe']

,startTime,geometry,fileID,sensor,cadence,cx,cy
0,2022-01-08 18:31:57+00:00,"POLYGON ((-178.57089 63.88048, -177.54237 65.6...",S1A_IW_SLC__1SDV_20220108T183157_20220108T1832...,sentinel1,NaT,-179.5,66.715039
1,2022-01-20 18:31:56+00:00,"POLYGON ((-178.57288 63.88073, -177.54451 65.6...",S1A_IW_SLC__1SDV_20220120T183156_20220120T1832...,sentinel1,11 days 23:59:59,-179.5,67.717565
0,2022-01-08 18:31:57+00:00,"POLYGON ((-178.57089 63.88048, -177.54237 65.6...",S1A_IW_SLC__1SDV_20220108T183157_20220108T1832...,sentinel1,NaT,-179.5,66.715039
1,2022-01-20 18:31:56+00:00,"POLYGON ((-178.57288 63.88073, -177.54451 65.6...",S1A_IW_SLC__1SDV_20220120T183156_20220120T1832...,sentinel1,11 days 23:59:59,-179.5,67.717565
0,2022-01-07 17:50:54+00:00,"POLYGON ((-168.35263 63.74250, -167.43529 65.3...",S1A_IW_SLC__1SDV_20220107T175054_20220107T1751...,sentinel1,NaT,-179.5,66.715039
...,...,...,...,...,...,...,...
1,2022-01-20 18:31:56+00:00,"POLYGON ((-178.57288 63.88073, -177.54451 65.6...",S1A_IW_SLC__1SDV_20220120T183156_20220120T1832...,sentinel1,11 days 23:59:59,-179.5,67.717565
0,2022-01-08 18:31:57+00:00,"POLYGON ((-178.57089 63.88048, -177.54237 65.6...",S1A_IW_SLC__1SDV_20220108T183157_20220108T1832...,sentinel1,NaT,-179.5,66.715039
0,2022-01-01 05:18:14+00:00,"POLYGON ((176.19861 71.76492, 177.80919 70.033...",S1A_IW_SLC__1SDH_20220101T051814_20220101T0518...,sentinel1,NaT,-179.5,66.715039
1,2022-01-13 05:18:13+00:00,"POLYGON ((176.19510 71.76494, 177.80624 70.033...",S1A_IW_SLC__1SDH_20220113T051813_20220113T0518...,sentinel1,11 days 23:59:59,-179.5,67.717565


In [19]:
continents['Europe']

,startTime,geometry,sensor,fileID,cadence,cx,cy
0,2022-01-02 11:47:28.291000+00:00,"POLYGON ((-17.20199 14.35393, -18.21865 14.368...",sentinel2,"HLS.S30.T28PBB.2022002T113501.v2.0,HLS.S30.T27...",NaT,-17.125043,14.797578
1,2022-01-03 11:27:20.964000+00:00,"POLYGON ((-17.20199 14.35393, -17.56130 14.359...",landsat8,"HLS.L30.T27PZS.2022003T112720.v2.0,HLS.L30.T28...",0 days 23:39:52.673000,-16.113716,11.790431
2,2022-01-05 19:18:09+00:00,"POLYGON ((-15.41438 13.52119, -17.69294 13.081...",sentinel1,S1A_IW_SLC__1SDV_20220105T191809_20220105T1918...,2 days 07:50:48.036000,-16.113716,12.792813
3,2022-01-07 11:47:22.657000+00:00,"POLYGON ((-17.20199 14.35393, -18.05469 14.366...",sentinel2,"HLS.S30.T28PBB.2022007T113349.v2.0,HLS.S30.T27...",1 days 16:29:13.657000,-16.113716,13.795196
4,2022-01-11 11:27:18.329000+00:00,"POLYGON ((-17.20199 14.35393, -17.55269 14.359...",landsat8,"HLS.L30.T27PZS.2022011T112718.v2.0,HLS.L30.T28...",3 days 23:39:55.672000,-16.113716,14.797578
...,...,...,...,...,...,...,...
13,2022-01-25 02:27:31+00:00,"POLYGON ((51.44460 10.62546, 51.81510 12.42926...",sentinel1,S1A_IW_SLC__1SDV_20220125T022731_20220125T0228...,1 days 19:31:00.407000,-15.102389,11.790431
14,2022-01-25 14:54:09+00:00,"POLYGON ((48.26905 12.42670, 48.63976 10.62221...",sentinel1,S1A_IW_SLC__1SDV_20220125T145409_20220125T1454...,0 days 12:26:38,-15.102389,12.792813
15,2022-01-26 07:16:11.200000+00:00,"POLYGON ((51.08927 10.76702, 50.08500 10.76568...",sentinel2,"HLS.S30.T39PVP.2022026T070139.v2.0,HLS.S30.T39...",0 days 16:22:02.200000,-15.102389,13.795196
16,2022-01-31 06:56:36.641000+00:00,"POLYGON ((51.08927 10.76702, 50.08500 10.76568...",landsat8,"HLS.L30.T39PVN.2022031T065636.v2.0,HLS.L30.T39...",4 days 23:40:25.441000,-15.102389,14.797578


In [9]:
continents['Europe'].cy.unique()

array([66.71503908, 67.71756486, 68.72009064, 65.71251331, 64.70998753,
        4.55844075,  2.55338919,  3.55591497])

In [ ]:
len(cont_points['Asia'])

In [ ]:
continents['Africa']

In [ ]:
continents['N_America']

In [ ]:
continents['S_America']

In [ ]:
continents['Oceania']

### Plotting functions

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.set_xlim(-180,180)
ax.set_ylim(-85,85)
europe_plot = df_europe_test.plot(column='av_cad_hours', ax=ax, legend=True)
df1.plot(column='av_cad_hours', ax = ax)
df2.plot(column='av_cad_hours', ax = ax)
df3.plot(column='av_cad_hours', ax = ax)
df4.plot(column='av_cad_hours', ax = ax)
df5.plot(column='av_cad_hours', ax = ax)
cx.add_basemap(ax=ax,crs='EPSG:4326',source=cx.providers.CartoDB.VoyagerNoLabels)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Global Coverage Cadence in Hours')